In [33]:
import logging
from pathlib import Path

import sys

sys.path.append("/home/xbankov/product_bundle/src/")

import pandas as pd
from bundling_content_based_filtering import encode_description
import networkx as nx
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from dataset import read_dataset, split
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

data_dir = Path("../data/data.csv")

logging.basicConfig(
    level=logging.INFO,  # Set the logging level (DEBUG, INFO, WARNING, ERROR, CRITICAL)
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
)
data = read_dataset(path=data_dir)
data = data.drop(["Quantity", "Country", "CustomerID"], axis=1)
data["InvoiceDate"] = pd.to_datetime(data["InvoiceDate"])
data["Description"] = data["Description"].fillna("")
data = encode_description(data)

In [34]:
data.nunique()

InvoiceNo      18405
ItemID          3659
Description     3871
InvoiceDate    17169
UnitPrice        356
dtype: int64

In [25]:
test =  read_dataset(path=data_dir)

In [28]:
test["InvoiceNo"].nunique()

18405

In [18]:
transactions = data.groupby("InvoiceNo")["ItemID"].apply(list).to_list()

1516

In [19]:
# Create a directed graph were nodes represent transactions
G = nx.DiGraph()

# Add transactions as nodes to the graph
for i, transaction in enumerate(transactions):
    G.add_node(i, products=transaction)

# Define relationships (edges) between transactions
# You can customize this logic based on your criteria (e.g., common products)
for i in range(len(transactions)):
    for j in range(i + 1, len(transactions)):
        common_products = set(transactions[i]) & set(transactions[j])
        if common_products:
            G.add_edge(i, j, common_products=len(common_products))

In [20]:
# Define node features (you can customize this based on your data)
# Here, we use the total number of products in each transaction as a feature
node_features = torch.tensor([len(G.nodes[i]['products']) for i in G.nodes()], dtype=torch.float32).unsqueeze(1)

In [22]:
node_features.shape

torch.Size([1516, 1])

In [14]:

# Split the data into train and test nodes
nodes = list(G.nodes())
train_nodes, test_nodes = train_test_split(nodes, train_size=0.8, random_state=42)

# Create adjacency matrix
adj_matrix = nx.to_numpy_matrix(G, nodelist=nodes)

# Define a simple GNN model
class GNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNN, self).__init__()
        self.gc1 = nn.Linear(input_dim, hidden_dim)
        self.gc2 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x, adj_matrix):
        x = self.relu(self.gc1(x))
        x = torch.matmul(adj_matrix, x)  # Aggregate information from connected transactions
        x = self.gc2(x)
        return x

# Convert node features to PyTorch tensor
node_features = torch.tensor(node_features, dtype=torch.float32)

# Create GNN model
input_dim = 1  # Only using the number of products as a feature
hidden_dim = 64
output_dim = 1  # We'll predict a single value (e.g., bundle score)
model = GNN(input_dim, hidden_dim, output_dim)

# Define loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Train the GNN on the training nodes
train_indices = [nodes.index(node) for node in train_nodes]
train_features = node_features[train_indices]
train_adj_matrix = adj_matrix[train_indices][:, train_indices]

for epoch in range(50):
    optimizer.zero_grad()
    outputs = model(train_features, torch.tensor(train_adj_matrix, dtype=torch.float32))
    loss = criterion(outputs, train_features)  # Reconstruction loss (minimize the difference)
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch + 1}/{50}, Loss: {loss.item()}')

# Now, you can use the trained GNN to recommend bundles for test nodes
test_indices = [nodes.index(node) for node in test_nodes]
test_features = node_features[test_indices]
test_adj_matrix = adj_matrix[test_indices][:, test_indices]

with torch.no_grad():
    test_recommendations = model(test_features, torch.tensor(test_adj_matrix, dtype=torch.float32)).numpy()


AttributeError: module 'networkx' has no attribute 'to_numpy_matrix'